In [1]:
import os

In [2]:
%pwd

'c:\\Users\\satya\\Documents\\data_science_roadmap\\dl_projects\\Chicken_disease\\Chicken-Disease-Classification\\reserach'

In [3]:
os.chdir("../")

In [4]:
%pwd #Project working directory

'c:\\Users\\satya\\Documents\\data_science_roadmap\\dl_projects\\Chicken_disease\\Chicken-Disease-Classification'

In [13]:
# Data Ingestion config/Entity which helps defining custom written type of function using entity
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from cnnClassifier.constants import *  #importing everything from the constant file 
from cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
          self,
          config_filepath=CONFIG_FILE_PATH,
          params_filepath= PARAMS_FILE_PATH):
          self.config=read_yaml(config_filepath)    # Read the data
          self.params=read_yaml(params_filepath)

          create_directories([self.config.artifacts_root])  #Note configbox is used here

    def get_data_ingestion_config(self) -> DataIngestionConfig:   #Used for data ingestion
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [16]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-08-03 00:24:05,788: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-03 00:24:05,789: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-03 00:24:05,790: INFO: common: created directory at: artifacts]
[2024-08-03 00:24:05,792: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-03 00:24:06,849: INFO: 222777804: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11298309
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "41f29cd968c219fd76250684ca30acede887fc760cfbd593cce2c20f17a9e63a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DE8C:1CCE70:135AF2:16373D:66ADBE9B
Accept-Ranges: bytes
Date: Sat, 03 Aug 2024 05:22:37 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdfw8210176-DFW
X-C